In [4]:
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

In [32]:
import pdfplumber
import pytesseract
import pandas as pd
from PIL import Image
import warnings
import openai
from glob import glob
from tqdm import tqdm
import os
import json
import re
import shutil
warnings.filterwarnings("ignore")

prompt = '''
    Imagine you are an intelligent assistant tasked with extracting all quantitative data from a construction project evaluation document that will be used to calculate key performance indicators (KPIs). Your role is to identify and list every numerical figure related to project budgets, change orders, compliance with procurement rules, construction management, transparency measures, and performance against bond criteria. Format your findings as follows: {"Project Name":xxx,"Document Type": Bid/Contract/ChangeOrder/RFP/RFQ, "Numerical Data": {"Construction Costs": "xx", "Budget Variance": "xx"}}, and continue for each section relevant to KPIs. Ensure to capture data from sections on Change Orders, Procurement Contractors, Construction Management, Overall Transparency, and Performance vs. Bond Criteria. All money should be present as value without comma and $. You must return a python Dict data structure and just return a dict, never use ```python``` or other description, and just return the dict data structure without any description.
'''

openai.api_key = "sk-a3b51790b3534859af1631d60338b2f2"
openai.api_base = "https://api.deepseek.com"

def get_info(page:str, prompt:str=''):
    response = openai.ChatCompletion.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": prompt },
            {"role": "user", "content": f"{page}"},
        ]
    )
    return response.choices[0].message['content']

def extract_text_and_images_from_pdf(pdf_path):
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            image = page.to_image(resolution=300).original 
            text = pytesseract.image_to_string(image)
            pages.append((image, text))
    return pages

pdf_paths = glob("/Users/moonliet/Downloads/drive-download-20240507T231719Z-001/*.pdf")
results = []
for pdf_path in tqdm(pdf_paths):
    pages = extract_text_and_images_from_pdf(pdf_path)
    page_str = "\n".join([page[1] for page in pages])
    res = get_info(page_str, prompt=prompt)
    results.append(res)


100%|██████████| 17/17 [11:48<00:00, 41.66s/it]


In [33]:
def find_longest_braces_content(text: str) -> str:

    brace_positions = [(m.start(0), m.end(0)) for m in re.finditer(r'\{|\}', text)]
    
    if not brace_positions:
        return ""  
    
    max_length = 0
    longest_content = ""


    for i in range(len(brace_positions)):
        for j in range(i + 1, len(brace_positions)):
            start, end = brace_positions[i][1], brace_positions[j][0]
            if text[brace_positions[i][0]] == '{' and text[brace_positions[j][0]] == '}':
                current_length = end - start
                if current_length > max_length:
                    max_length = current_length
                    longest_content = text[start:end]

    return longest_content

In [34]:
import re
import ast

def extract_dict_from_code_block(text: str) -> dict:
    pattern = r'```(json|python)\s*\{[\s\S]*?\}\s*```'
    match = re.search(pattern, text)
    
    if match:
        code_block = match.group(0)
        json_str = re.sub(r'```(json|python)\s*|\s*```', '', code_block)
        
        try:
            dictionary = ast.literal_eval(json_str)
            if isinstance(dictionary, dict):
                return dictionary
        except (SyntaxError, ValueError):
            pass
    
    return {}

In [35]:
clean_results = []
for i in range(len(results)):
    try:
        result = eval(results[i])
        if isinstance(result, dict): 
            clean_results.append(result)
    except:
        if "```" in results[i]:
            extracted_dict = extract_dict_from_code_block(results[i])
            if isinstance(extracted_dict, dict): 
                clean_results.append(extracted_dict)


In [36]:
len(clean_results)

16

In [37]:
clean_results

[{'Project Name': 'Fallbrook Union High School District',
  'Document Type': 'Bond',
  'Numerical Data': {'Bond Proceeds': '9643557.50',
   'Building Fund': '8782000.00',
   'Debt Service Fund': '643557.50',
   'Cost of Issuance': '155000.00',
   "Underwriter's Discount": '63000.00',
   'Total Interest': '4926781.43',
   'Net Interest': '4346223.93',
   'Total Debt Service': '13926781.43',
   'Maximum Annual Debt Service': '1554062.50',
   'Average Annual Debt Service': '564535.67',
   'Par Amount': '9000000.00',
   'Premium': '643557.50',
   'Net Proceeds': '9580557.50',
   'Net Debt Service': '13283223.93'}},
 {'Project Name': 'Fallbrook Union High School District',
  'Document Type': 'Master Agreement for Architectural Services',
  'Numerical Data': {'Construction Costs': 'Not specified',
   'Budget Variance': 'Not specified'}},
 {'Project Name': 'Fallbrook Union High School District Emergency Waiver',
  'Document Type': 'Contract',
  'Numerical Data': {'Construction Costs': '45000'

In [38]:
fallbrook_documents =[]
for sub in clean_results:
    if "Project Name" not in sub.keys():
        continue
    if "fallbrook" in sub['Project Name'].lower():
        fallbrook_documents.append(sub)
def calculate_kpi_1(documents):
    total_construction_costs = 0
    total_budget = 0
    
    for doc in documents:
        try:
            total_construction_costs += float(doc['Numerical Data'].get('Construction Costs', 0))
            total_budget += float(doc['Numerical Data'].get('Construction Budget', 0))
        except ValueError:
            continue
            
    if total_budget == 0 or total_construction_costs == 0:
        return "Needs Improvement"
    
    ratio = (total_construction_costs / total_budget) * 100
    if ratio >= 80:
        return "Exceeds Standards"
    elif 70 <= ratio < 80:
        return "At Standard"
    else:
        return "Needs Improvement"

def calculate_kpi_2(documents):
    total_change_order_amount = 0
    total_construction_costs = 0
    
    for doc in documents:
        try:
            total_change_order_amount += float(doc['Numerical Data'].get('Change Order Amount', 0))
            total_construction_costs += float(doc['Numerical Data'].get('Construction Costs', 0))
        except ValueError:
            continue
            
    if total_construction_costs == 0:
        return "Needs Improvement"
    
    change_order_rate = (total_change_order_amount / total_construction_costs) * 100
    if change_order_rate < 5:
        return "Exceeds Standards"
    elif 5 <= change_order_rate < 8:
        return "At Standard"
    else:
        return "Needs Improvement"

kpi_1_result = calculate_kpi_1(fallbrook_documents)
kpi_2_result = calculate_kpi_2(fallbrook_documents)

kpi_results = {
    'Project Name': 'Fallbrook High School',
    'KPI #1': kpi_1_result,
    'KPI #2': kpi_2_result
}

print(kpi_results)


{'Project Name': 'Fallbrook High School', 'KPI #1': 'Needs Improvement', 'KPI #2': 'Exceeds Standards'}


In [39]:
prompt = """
You are a construction project analyst tasked with evaluating various Key Performance Indicators (KPIs) for the Fallbrook High School construction project. Based on the provided project documents, please analyze and provide detailed responses for each KPI listed below. Follow the specific output format strictly.

KPI #3: Procurement Contracts & Professional Services
- Did the procurement contracts comply with public contract code provisions?
- Did the procurement contracts clearly define the scope and evaluation criteria for consultant selection?
- Was there widespread advertisement and outreach for consultant selection?
- Was there an independent evaluation committee (outside/in-house) recommending consultants to the superintendent/chancellor?
- Was sole source procurement clearly justified for cost-effectiveness and consistent with applicable code provisions?
- Was consultant expertise thoroughly evaluated?
- Were projects rebid for low number of bidders or costs 10% or more in excess of the engineer's estimate?

KPI #4: Construction Management
- Were project managers responsible for projects at 1 or 2 schools with a construction value of $20M or less?
- Were full-time project managers assigned to major district-wide projects?
- Were consultant/management companies assigned to assist in-house project managers as needed?

KPI #5: Overall Transparency
- Were written or oral reports provided to the governing board?
- Was there independent and open recruitment and evaluation for qualified oversight members?
- Were recommendations for the selection of members made by a committee independent of the superintendent or board members?
- Did the oversight committee set the agenda and number of meetings per year?
- Did minutes or reports to the governing board indicate robust probing discussion of proper spending of funds?
- Did the oversight committee provide input into the scope of the performance audit?

KPI #6: Performance vs. Bond Criteria
- Was a Facilities Master Plan prepared that included a Facilities Condition Assessment, Enrollment Projections, and Projections of Future Facilities Requirements by campus?
- Were project lists clearly laid out in the ballot measure by campus, or were appropriate system-wide improvements clearly identified?
- Did the district have clear system-wide policies and priorities identified?
- Were project lists in the bond measure backed with detailed cost estimates that coincided with the amount of debt authorized?
- Did district spending align with the ballot measure?

Please provide the analysis and rating for each KPI in the following format:

KPI #3: Procurement Contracts & Professional Services
- Compliance: [Yes/No]
- Scope and Evaluation Criteria: [Yes/No]
- Advertisement and Outreach: [Yes/No]
- Independent Evaluation Committee: [Yes/No]
- Sole Source Justification: [Yes/No]
- Consultant Expertise Evaluation: [Yes/No]
- Project Rebid: [Yes/No]

KPI #4: Construction Management
- Project Manager Responsibility: [Yes/No]
- Full-time PM for District-wide Projects: [Yes/No]
- Consultant/Management Companies Assistance: [Yes/No]

KPI #5: Overall Transparency
- Reports to Governing Board: [Yes/No]
- Independent Recruitment and Evaluation: [Yes/No]
- Independent Committee Recommendations: [Yes/No]
- Oversight Committee Sets Agenda: [Yes/No]
- Probing Discussion in Minutes: [Yes/No]
- Oversight Committee Input into Audit Scope: [Yes/No]

KPI #6: Performance vs. Bond Criteria
- Facilities Master Plan: [Yes/No]
- Project List in Ballot Measure: [Yes/No]
- System-wide Policies and Priorities: [Yes/No]
- Detailed Cost Estimates: [Yes/No]
- Spending Consistency with Ballot Measure: [Yes/No]

"""

other_kpi_output = get_info(fallbrook_documents,prompt=prompt)

In [41]:
def extract_kpi_info(output):
    kpi_info = {}
    
    kpi_patterns = {
        "KPI #3": r"KPI #3: Procurement Contracts & Professional Services\n((?:- .+?: .+\n)+)",
        "KPI #4": r"KPI #4: Construction Management\n((?:- .+?: .+\n)+)",
        "KPI #5": r"KPI #5: Overall Transparency\n((?:- .+?: .+\n)+)",
        "KPI #6": r"KPI #6: Performance vs. Bond Criteria\n((?:- .+?: .+\n)+)"
    }
    
    for kpi, pattern in kpi_patterns.items():
        match = re.search(pattern, output)
        if match:
            kpi_info[kpi] = match.group(1).strip()

    return kpi_info


def evaluate_kpi_3(kpi_info):
    items = kpi_info.split('\n')
    yes_count_1_2 = sum(1 for item in items[:2] if 'Yes' in item)
    yes_count_3_9 = sum(1 for item in items[2:] if 'Yes' in item)
    total_count_3_9 = len(items[2:])
    
    if yes_count_1_2 == 2 and yes_count_3_9 / total_count_3_9 >= 0.9:
        return "Exceeds Standards"
    elif yes_count_1_2 == 2 and ((yes_count_3_9 >= 5 and yes_count_3_9 / total_count_3_9 >= 0.9) or (yes_count_3_9 >= 1 and yes_count_3_9 / total_count_3_9 >= 0.8)):
        return "At Standard"
    elif yes_count_1_2 >= 1 and yes_count_3_9 >= 3 and yes_count_3_9 / total_count_3_9 >= 0.8:
        return "Needs Improvement"
    else:
        return "Not Certifiable"

def evaluate_kpi_4(kpi_info):
    items = kpi_info.split('\n')
    yes_count = sum(1 for item in items if 'Yes' in item)
    
    if yes_count == 3:
        return "Exceeds Standards"
    elif yes_count == 2:
        return "At Standard"
    elif yes_count == 1:
        return "Needs Improvement"
    else:
        return "Not Certifiable"

def evaluate_kpi_5(kpi_info):
    items = kpi_info.split('\n')
    yes_count = sum(1 for item in items if 'Yes' in item)
    
    if yes_count == 6:
        return "Exceeds Standards"
    elif yes_count == 4:
        return "At Standard"
    elif yes_count <= 3:
        return "Needs Improvement"
    else:
        return "Not Certifiable"

def evaluate_kpi_6(kpi_info):
    items = kpi_info.split('\n')
    yes_count = sum(1 for item in items if 'Yes' in item)
    
    if yes_count == 5:
        return "Exceeds Standards"
    elif yes_count == 4:
        return "At Standard"
    elif yes_count <= 3:
        return "Needs Improvement"
    else:
        return "Not Certifiable"


def calculate_overall_rating(kpi_levels):
    rating_counts = {
        "Exceeds Standards": 0,
        "At Standard": 0,
        "Needs Improvement": 0,
        "Not Certifiable": 0
    }
    
    for level in kpi_levels.values():
        rating_counts[level] += 1
    
    if rating_counts["Exceeds Standards"] >= 3:
        return "Outstanding"
    elif rating_counts["At Standard"] >= 3 or (rating_counts["At Standard"] >= 2 and rating_counts["Needs Improvement"] >= 1):
        return "Excellent"
    elif rating_counts["Needs Improvement"] >= 3 or (rating_counts["Needs Improvement"] >= 2 and rating_counts["At Standard"] >= 1):
        return "Good"
    else:
        return "Not Certifiable"


def integrate_kpi_results(kpi_results, other_kpi_info):
    kpi_levels = {}
    
    if "KPI #3" in other_kpi_info:
        kpi_levels["KPI #3"] = evaluate_kpi_3(other_kpi_info["KPI #3"])
    if "KPI #4" in other_kpi_info:
        kpi_levels["KPI #4"] = evaluate_kpi_4(other_kpi_info["KPI #4"])
    if "KPI #5" in other_kpi_info:
        kpi_levels["KPI #5"] = evaluate_kpi_5(other_kpi_info["KPI #5"])
    if "KPI #6" in other_kpi_info:
        kpi_levels["KPI #6"] = evaluate_kpi_6(other_kpi_info["KPI #6"])
    
    overall_rating = calculate_overall_rating(kpi_levels)
    kpi_results.update(kpi_levels)
    kpi_results["Overall"] = overall_rating
    return kpi_results


other_kpi_info = extract_kpi_info(other_kpi_output)


kpi_results = {
    'Project Name': 'Fallbrook High School',
    'KPI #1': 'Needs Improvement',
    'KPI #2': 'Exceeds Standards'
}


kpi_results = integrate_kpi_results(kpi_results, other_kpi_info)


print(json.dumps(kpi_results, indent=2))

{
  "Project Name": "Fallbrook High School",
  "KPI #1": "Needs Improvement",
  "KPI #2": "Exceeds Standards",
  "KPI #3": "Not Certifiable",
  "KPI #4": "Not Certifiable",
  "KPI #5": "Needs Improvement",
  "KPI #6": "Needs Improvement",
  "Overall": "Not Certifiable"
}


In [42]:
with open('/Users/moonliet/Downloads/capstoneout/results.json', 'w') as f:
    json.dump(clean_results, f)

print("Data extraction and saving to file complete.")


Data extraction and saving to file complete.
